In [1]:
from typing import List, Dict, Any
import json


def load_records_from_coco_json(coco_json_path: str, image_root: str) -> List[Dict[str, Any]]:
    """Convert a tiny subset of COCO-format json into our records format.

    NOTE: This is a convenience helper and not a full COCO parser.
    """
    with open(coco_json_path, 'r', encoding='utf-8') as f:
        coco = json.load(f)

    images = {img['id']: img for img in coco.get('images', [])}
    ann_by_image = {}
    for ann in coco.get('annotations', []):
        img_id = ann['image_id']
        ann_by_image.setdefault(img_id, []).append(ann)

    records = []
    for img_id, img in images.items():
        anns = ann_by_image.get(img_id, [])
        bboxes = []
        classes = []
        for a in anns:
            x, y, w, h = a['bbox']
            bboxes.append([x, y, x + w, y + h])
            classes.append(a['category_id'])
        records.append({'image_path': f"{image_root}/{img['file_name']}", 'bboxes': bboxes, 'classes': classes})
    return records